# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
5.5193e18,"""../../../../data/SampleHDS.jso…","""3258165678962932075""","""9184493603045775258""","""7665004586841417428""","""16521606622963095632""",false
1.5780e19,"""../../../../data/SampleHDS.jso…","""3380990991682986277""","""501112477279658905""","""7665004586841417428""","""1065193309851533591""",true
1.3277e19,"""../../../../data/SampleHDS.jso…","""1371915189835564343""","""9287635948254952966""","""3266902106700833942""","""16521606622963095632""",false
1.3712e19,"""../../../../data/SampleHDS.jso…","""3380990991682986277""","""635234052266619310""","""12538307450235557520""","""16521606622963095632""",false
1.1502e19,"""../../../../data/SampleHDS.jso…","""3258165678962932075""","""4067609721217044417""","""12538307450235557520""","""8850363451957809979""",true
4.5287e17,"""../../../../data/SampleHDS.jso…","""1371915189835564343""","""395630959096225915""","""9534268819796278518""",null,false
1.1111e19,"""../../../../data/SampleHDS.jso…","""1371915189835564343""","""2763231852670596127""","""12538307450235557520""","""8850363451957809979""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
8.0678e18,"""../../../../data/SampleHDS.jso…","""595426668990621478""","""4788707918917151690""","""FirstMortgage30yr""","""10632139373496480660""","""Rejected"""
6.1752e18,"""../../../../data/SampleHDS.jso…","""6617398294627908307""","""12127642948343536515""","""FirstMortgage30yr""","""1704931731693259024""","""Accepted"""
2.5197e18,"""../../../../data/SampleHDS.jso…","""745765905844617548""","""15327258071907462668""","""MoneyMarketSavingsAccount""","""10632139373496480660""","""Rejected"""
3.3325e18,"""../../../../data/SampleHDS.jso…","""6617398294627908307""","""17862234441801319459""","""BasicChecking""","""10632139373496480660""","""Rejected"""
1.6156e19,"""../../../../data/SampleHDS.jso…","""595426668990621478""","""15071501714278042604""","""BasicChecking""","""17310792191252599422""","""Accepted"""
6.5497e18,"""../../../../data/SampleHDS.jso…","""745765905844617548""","""14038888645691559395""","""UPlusFinPersonal""",null,"""Rejected"""
7.3540e18,"""../../../../data/SampleHDS.jso…","""745765905844617548""","""15396348283890255518""","""BasicChecking""","""17310792191252599422""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.3491e19,"""../../../../data/SampleHDS.jso…","""2692055695808773939""","""4491411939902417221""","""FirstMortgage30yr""","""1387400346206562235""","""Rejected"""
1.1546e19,"""../../../../data/SampleHDS.jso…","""13410253385430347817""","""8058086072564974119""","""FirstMortgage30yr""","""7150507913670653177""","""Accepted"""
6.0001e17,"""../../../../data/SampleHDS.jso…","""13761351380390763994""","""6890007786761513213""","""MoneyMarketSavingsAccount""","""1387400346206562235""","""Rejected"""
9.9904e18,"""../../../../data/SampleHDS.jso…","""13410253385430347817""","""12045105415383662377""","""BasicChecking""","""1387400346206562235""","""Rejected"""
2.7202e18,"""../../../../data/SampleHDS.jso…","""2692055695808773939""","""14682893089185261092""","""BasicChecking""","""5539393932290085743""","""Accepted"""
1.4746e19,"""../../../../data/SampleHDS.jso…","""13761351380390763994""","""629733628035878037""","""UPlusFinPersonal""",null,"""Rejected"""
7.9404e18,"""../../../../data/SampleHDS.jso…","""13761351380390763994""","""12141447734366352940""","""BasicChecking""","""5539393932290085743""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.7409e19,"""9482121276729955857""","""13769156743135976308""","""FirstMortgage30yr""","""16098889049692337928""","""Rejected"""
1.3571e18,"""7365023927319990261""","""1260274798180866945""","""FirstMortgage30yr""","""367488105307861947""","""Accepted"""
2.6840e17,"""6003827832131045698""","""5594234946323233552""","""MoneyMarketSavingsAccount""","""16098889049692337928""","""Rejected"""
1.3082e19,"""7365023927319990261""","""11834038957890011786""","""BasicChecking""","""16098889049692337928""","""Rejected"""
5.3528e18,"""9482121276729955857""","""3915947970818609058""","""BasicChecking""","""798190143383803654""","""Accepted"""
1.1346e18,"""6003827832131045698""","""13498309582079812856""","""UPlusFinPersonal""",null,"""Rejected"""
3.0766e18,"""6003827832131045698""","""12204910137169049533""","""BasicChecking""","""798190143383803654""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'